In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


# Machine Learning

## OpenVINO Tensorflow Model

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from openvino.runtime import Core

from IPython.display import Markdown, display

In [ ]:
# Some Basic Parameters for housekeeping

# set location of input files:
inpDir = os.path.join('./')

# set location of inputs for this module
moduleDir = './'

# set location of inputs for this module
subDir = 'vino'

# set location of output files
outDir = os.path.join('./')

# set location of model files
modelDir = os.path.join('./')

# define and set random state 
RANDOM_STATE = 24
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

# parameters for Matplotlib
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 9),
          'axes.labelsize': 'large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'large',
          'savefig.dpi': 75,
          'image.interpolation': 'none',
          'savefig.bbox' : 'tight',
          'lines.linewidth' : 1,
          'legend.numpoints' : 1
         }

CMAP = plt.cm.brg
plt.rcParams.update(params);
plt.set_cmap(CMAP);

plt.style.use('bmh')

TEST_SIZE = 0.2

In [ ]:
base_model_dir = os.path.join(modelDir, 'flower_photos')# expand the path as per the os and environment

model_name = "dropout_bn_no_bias_swish_flowers"
model_name = f'{model_name}.h5'
model_path = os.path.join(base_model_dir, model_name)

In [ ]:
## Load model
model = tf.keras.models.load_model(model_path)

model.summary()

In [ ]:
tf.saved_model.save(model,base_model_dir)

In [ ]:
print(model.layers[0].name, model.layers[0].get_input_shape_at(0))
print(model.layers[-1].name, model.layers[-1].get_output_at(0).name)

In [ ]:
# Construct the command for Model Optimizer.
mo_command = f'mo --saved_model_dir ../models/flower_photos/  --input_shape=\[1,188,188,3\]'
                 
mo_command = " ".join(mo_command.split())

print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

In [ ]:
mo_result = %sx $mo_command
print("\n".join(mo_result))

In [ ]:
ie = Core()

model = ie.read_model(model='saved_model.xml')
compiled_model = ie.compile_model(model=model, device_name='CPU')

input_layer_ir = compiled_model.input(0)
output_layer_ir = compiled_model.output('dense_1')

In [ ]:
input_layer_ir, output_layer_ir

## Load an Image

In [ ]:
# Text detection models expect an image in BGR format.
image = cv2.imread(filename=os.path.join(inpDir,'flower_photos/roses/15277801151_5ed88f40f0_n.jpg'))

image.shape

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB));

In [ ]:
input_layer_ir.shape

In [ ]:
# N,C,H,W = batch size, number of channels, height, width.
N, H, W, C  = input_layer_ir.shape
N, H, W, C

In [ ]:
# Resize the image to meet network expected input sizes.
resized_image = cv2.resize(image, (W, H))
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB));

In [ ]:
input_image = np.expand_dims(resized_image, 0)
input_image.shape

## Inference

In [ ]:
# Create an inference request.
y_pred = compiled_model([input_image])[output_layer_ir]
y_pred

In [ ]:
y_pred.argmax()

In [ ]:
flower_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
res = flower_names[y_pred.argmax()]
res